In [73]:
import pandas as pd
import numpy as np
# import scipy.stat as sps
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter

In [74]:
%%time
df = pd.read_excel('card transactions.xlsx')

Wall time: 9.67 s


In [75]:
# summary
numrecords = len(df)
stat_df = pd.DataFrame(df.isnull().sum(axis = 0))
stat_df = stat_df.reset_index().rename(columns = {'index':'variable', 0 :"# NaN"})
stat_df["% populated"] = (1 - stat_df["# NaN"]/numrecords)*100
stat_df["# unique"] = ''

for i in range(len(list(df))):
    stat_df["# unique"][i] = df[list(df)[i]].nunique()
    
stat_df

C:\Users\beizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,variable,# NaN,% populated,# unique
0,Recnum,0,100.000000,96753
1,Cardnum,0,100.000000,1645
2,Date,0,100.000000,365
3,Merchnum,3375,96.511736,13091
4,Merch description,0,100.000000,13126
5,Merch state,1195,98.764896,227
6,Merch zip,4656,95.187746,4567
7,Transtype,0,100.000000,4
8,Amount,0,100.000000,34909
9,Fraud,0,100.000000,2


In [76]:
# df.head(10)

In [77]:
##Identify any exclusions, bad records

# Remove single large transaction outlier in Amount
df = df[df['Amount']<3000000]

# Remove all but P transaction type
df = df[df['Transtype'] == 'P']

len(df)

96397

In [78]:
# --------------- Fill in NAs ----------

### Merchnum 
# Transform 0 values into NA
df.loc[df['Merchnum'] == '0', 'Merchnum'] = np.NaN

# fill most frequenct merchnum using merch description
dict1 = df.set_index('Merch description')['Merchnum'].to_dict()
df['Merchnum'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill most frequenct merchnum in this zip
dict1 = dict(zip(df['Merch zip'].dropna(), df['Merchnum']))
df['Merchnum'].fillna(df['Merch zip'].map(dict1), inplace = True)

pd.isnull(df['Merchnum']).sum()


1414

In [79]:
### Merch state and zip
numrecords - (df['Merch state'].isna() == df['Merch zip'].isna()).sum()


3810

In [80]:
### Merch zip
# df.groupby(['Merch description','Merch zip']).size()

# fill NA in Merch zip using Merch description
dict1 = df.set_index('Merch description')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch zip using Merchnum
dict1 = df.set_index('Merchnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch zip using Cardnum
dict1 = df.set_index('Cardnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Cardnum'].map(dict1), inplace = True)
# It will automatically choose zip with highest frequency

pd.isnull(df['Merch zip']).sum()


543

In [81]:
dic = dict(zip(df['Cardnum'], df['Merch zip']))
dic[5142310525]

60108.0

In [82]:
### Merch state
# df.groupby(['Merch description','Merch state']).size()
# Each Merch description has one matched state

# fill NA in Merch state using Merch description
dict1 = df.set_index('Merch description')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch state using Merchnum
dict1 = df.set_index('Merchnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch state using Merch zip

dict1 = dict(zip(df['Merch zip'].dropna(), df['Merch state']))
df['Merch state'].fillna(df['Merch zip'].map(dict1), inplace = True)

# fill NA in Merch state using Cardnum
dict1 = df.set_index('Cardnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Cardnum'].map(dict1), inplace = True)

pd.isnull(df['Merch state']).sum()


122

In [83]:
# Drop remained NA in three variables
df = df.dropna()
len(df)

94631

In [84]:
df.groupby(['Cardnum','Date'])['Amount'].value_counts()
# df[df['Cardnum'] == 5142190439]

Cardnum     Date        Amount 
5142110081  2010-11-26  636.20     1
            2010-12-27  495.90     1
5142110402  2010-01-19  506.50     1
            2010-01-31  20.81      1
            2010-02-03  30.26      1
            2010-02-04  54.00      1
                        437.76     1
5142110434  2010-10-06  1551.64    1
5142110749  2010-04-19  62.94      1
            2010-06-13  794.85     1
5142110909  2010-01-17  351.24     1
            2010-01-20  64.40      1
            2010-02-09  301.79     1
            2010-03-21  128.00     1
            2010-03-22  90.99      1
                        766.88     1
            2010-04-05  435.47     1
            2010-04-12  30.40      1
                        161.25     1
            2010-04-13  809.45     1
            2010-05-18  79.80      2
            2010-06-12  288.00     1
            2010-07-07  1535.40    1
            2010-07-12  51.75      1
            2010-12-20  107.76     1
            2010-12-27  305.25     1
      

In [85]:
# --------------- Create variables ----------
### Amount expert variables

# Cardnum
# sum the amount at the same day first and then rolling
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')



In [86]:
# (closed = 'left') in rolling function means when calculating past n days we exlude today's transactions

def amount_mean_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').mean(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

# def amount_max_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).max(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

# def amount_median_func(column, window, rsuffix):
#     dataframe = pd.DataFrame()
#     dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window).median(), on = [column,'Date'], rsuffix = rsuffix)
#     return dataframe

def amount_sum_func(column, window, rsuffix):
    dataframe = pd.DataFrame()
    dataframe = new_df.join(df2.groupby([column])['Amount'].rolling(window = window, closed = 'left').sum(), on = [column,'Date'], rsuffix = rsuffix)
    return dataframe

In [87]:
df2 = df.groupby(['Cardnum','Date'])['Amount'].sum().to_frame().reset_index().set_index('Date')
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(),on = ['Cardnum','Date'], rsuffix = '_cn_actual')

new_df = amount_mean_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_avg')
# new_df = amount_max_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_max')
# new_df = amount_median_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_median')
new_df = amount_sum_func(column = 'Cardnum',window = '1D',rsuffix = '_cn_1d_sum')
new_df['Amount_cn_1d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act-median'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_1d_median']
new_df['Amount_cn_1d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_avg']
# new_df['Amount_cn_1d_act/max'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_max']
new_df['Amount_cn_1d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_sum']
# new_df['Amount_cn_1d_act/median'] = new_df['Amount_cn_actual']/new_df['Amount_cn_1d_median']



In [88]:
new_df = amount_mean_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '4D',rsuffix = '_cn_4d_sum')
new_df['Amount_cn_4d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_avg']
new_df['Amount_cn_4d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_4d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '7D',rsuffix = '_cn_7d_sum')
new_df['Amount_cn_7d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/avg'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_avg']
new_df['Amount_cn_7d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_7d_sum']

new_df = amount_mean_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_avg')
new_df = amount_sum_func(column = 'Cardnum',window = '14D',rsuffix = '_cn_14d_sum')
new_df['Amount_cn_14d_act-avg'] = new_df['Amount_cn_actual'] - new_df['Amount_cn_14d_avg']
new_df['Amount_cn_14d_act/sum'] = new_df['Amount_cn_actual']/new_df['Amount_cn_14d_sum']

In [89]:
list(new_df)

['Recnum',
 'Cardnum',
 'Date',
 'Merchnum',
 'Merch description',
 'Merch state',
 'Merch zip',
 'Transtype',
 'Amount',
 'Fraud',
 'Amount_cn_actual',
 'Amount_cn_1d_avg',
 'Amount_cn_1d_sum',
 'Amount_cn_1d_act-avg',
 'Amount_cn_1d_act/avg',
 'Amount_cn_1d_act/sum',
 'Amount_cn_4d_avg',
 'Amount_cn_4d_sum',
 'Amount_cn_4d_act-avg',
 'Amount_cn_4d_act/avg',
 'Amount_cn_4d_act/sum',
 'Amount_cn_7d_avg',
 'Amount_cn_7d_sum',
 'Amount_cn_7d_act-avg',
 'Amount_cn_7d_act/avg',
 'Amount_cn_7d_act/sum',
 'Amount_cn_14d_avg',
 'Amount_cn_14d_sum',
 'Amount_cn_14d_act-avg',
 'Amount_cn_14d_act/avg',
 'Amount_cn_14d_act/sum']

In [90]:
new_df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Amount_cn_7d_avg,Amount_cn_7d_sum,Amount_cn_7d_act-avg,Amount_cn_7d_act/avg,Amount_cn_7d_act/sum,Amount_cn_14d_avg,Amount_cn_14d_sum,Amount_cn_14d_act-avg,Amount_cn_14d_act/avg,Amount_cn_14d_act/sum
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
